# Exercise 04. Enrichment and transformations

#### - Turn-in directory: `ex04/`.
#### - Files to turn in: `enrichment.ipynb`.
#### - Allowed functions: `import pandas as pd`, `import numpy as np`, `import requests`.

In [1]:
import pandas as pd
import numpy as np
import requests

####
## 1. Read the JSON file that you saved in ex02:
   - One of the columns is a float, so let's define its format in Pandas using `pd.options.display.float_format`: floats should be displayed with two decimals.
   - There are missing values from the Model; do not do anything with them.

In [2]:
pd.options.display.float_format = '{:.2f}'.format
df = pd.read_json('../data/auto.json', dtype={'Model': object})
df[~df['Model'].str.contains('[a-zA-Z]', na=False)]

,CarNumber,Refund,Fines,Make,Model
26,M5039X197RUS,2,7400.00,Volkswagen,None
81,M589CH197RUS,2,7900.00,Volkswagen,None
130,H837YK197RUS,2,4200.00,Audi,None
157,E316EH197RUS,1,1300.00,Volkswagen,None
333,X023HY197RUS,2,10200.00,Volvo,None
388,X023HY197RUS,2,6800.00,Volvo,None
410,Y687HM197RUS,1,8594.59,BMW,None
526,Y693HM197RUS,2,6500.00,BMW,None
576,96907X197RUS,2,3000.00,BMW,None


####
## 2. Enrich the dataframe using a sample from that dataframe:
   - Create a sample with 200 new observations using `random_state = 21`.
     - The sample should not contain new combinations of the `car number`, `make`, and `model`, so the entire dataset will be consistent in this regard.
     - There are no restrictions on the `refund` and `fines` columns. You can randomly select a value from these columns and apply it to any car number.
   - Concatenate the sample with the initial dataframe to create a new dataframe, `concat_rows`.

In [3]:
data_generated = df.sample(n=200, replace=True, random_state=21)
data_generated['Refund'] = df['Refund'].sample(n=200, replace=True, random_state=21).values
data_generated['Fines'] = df['Fines'].sample(n=200, replace=True, random_state=21).values
concat_rows = pd.concat([df, data_generated], ignore_index=True)
concat_rows.tail(210)

,CarNumber,Refund,Fines,Make,Model
715,O136HO197RUS,2,7800.00,Toyota,Corolla
716,O68897197RUS,2,12300.00,Ford,Focus
717,O718MM163RUS,2,8594.59,Ford,Focus
718,7065C8197RUS,2,11400.00,Volkswagen,Passat
719,O22097197RUS,1,24300.00,Ford,Focus
...,...,...,...,...,...
920,M942OT152RUS,1,2000.00,Ford,Focus
921,Y187O8161RUS,2,400.00,Ford,Focus
922,7064C8197RUS,1,12800.00,Volkswagen,Passat
923,8437XX154RUS,2,800.00,Ford,Focus


####
## 3. Enrich the `concat_rows` dataframe with a new column containing generated data.
   - Create a series named "Year" with random integers from 1980 to 2019.
   - Use `np.random.seed(21)` before generating the years.
   - Concatenate the series with the data frame and name it `fines`.

In [4]:
np.random.seed(21)
year = pd.Series(np.random.randint(1980, 2020, size=len(concat_rows)), name='Year', index=concat_rows.index)
fines = pd.concat([concat_rows, year], axis=1)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,M942OT152RUS,1,2000.00,Ford,Focus,1981
921,Y187O8161RUS,2,400.00,Ford,Focus,1992
922,7064C8197RUS,1,12800.00,Volkswagen,Passat,2007
923,8437XX154RUS,2,800.00,Ford,Focus,2005


In [5]:
fines.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
Year         925
dtype: int64

####
## 4. Enrich the dataframe with data from another dataframe:
   - Create a new dataframe with car numbers and their owners.
      - Get the most popular surnames in the US (you can find the file [surname.json](datasets/surname.json) in the folder datasets).
      - Create a new series with the surnames. They should not contain special characters, such as commas or brackets. The count should equal the number of unique car numbers in the sample (use `random_state = 21`).

In [6]:
surnames = pd.read_json('../data/surname.json')
surnames = surnames.iloc[1:, 0]
surnames.name = 'SURNAMES'
surnames = surnames.drop_duplicates()
surnames = surnames.replace('[^a-zA-Z ]', '', regex=True).str.strip()
surnames = surnames[surnames != '']
surnames = pd.Series(surnames)
surnames


1         ADAMS
2         ALLEN
3       ALVAREZ
4      ANDERSON
5        BAILEY
         ...   
96     WILLIAMS
97       WILSON
98         WOOD
99       WRIGHT
100       YOUNG
Name: SURNAMES, Length: 100, dtype: object

In [7]:
unique_carnumbers = fines['CarNumber'].unique()
unique_carnumbers = pd.Series(unique_carnumbers)
print(unique_carnumbers)
surnames = surnames.sample(n=len(unique_carnumbers), replace=True, random_state=21, ignore_index=True)
surnames

0      Y163O8161RUS
1       E432XX77RUS
2       7184TT36RUS
3      X582HE161RUS
4      92918M178RUS
           ...     
526    O136HO197RUS
527    O22097197RUS
528    M0309X197RUS
529    O673E8197RUS
530    8610T8154RUS
Length: 531, dtype: object


0      RICHARDSON
1            ROSS
2          MORGAN
3          BAILEY
4           LOPEZ
          ...    
526      CAMPBELL
527          HALL
528         BAKER
529          DIAZ
530        MORGAN
Name: SURNAMES, Length: 531, dtype: object

####
## - Create the dataframe `owners` with two columns: `CarNumber` and `SURNAME`.

In [8]:
owners = pd.DataFrame({
    'CarNumber': unique_carnumbers,
    'SURNAME': surnames
})

owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


####
## - Append five more observations to the `fines` dataframe. Come up with your own ideas for `CarNumber`, etc.

In [9]:
five_new_carnumbers = []
# for i in range(5):
#    five_new_carnumbers.append(f"PLATE_{i}_RUS")

add_five = fines.sample(n=5, replace=True, random_state=21).copy()

# add_five['CarNumber'] = five_new_carnumbers
add_five['CarNumber'] = pd.NA

np.random.seed(21)
add_five['Year'] = np.random.randint(2020, 2026, size=len(add_five))
add_five['Refund'] = fines['Refund'].sample(n=len(add_five), random_state=21).values
add_five['Fines'] = fines['Fines'].sample(n=len(add_five), random_state=21).values
fines = pd.concat([fines, add_five], ignore_index=True)

fines.tail(10)

,CarNumber,Refund,Fines,Make,Model,Year
920,M942OT152RUS,1,2000.00,Ford,Focus,1981
921,Y187O8161RUS,2,400.00,Ford,Focus,1992
922,7064C8197RUS,1,12800.00,Volkswagen,Passat,2007
923,8437XX154RUS,2,800.00,Ford,Focus,2005
924,C410X938RUS,2,2200.00,Ford,Focus,1997
925,NaN,1,500.00,Ford,Focus,2021
926,NaN,2,15200.00,Ford,Focus,2020
927,NaN,1,500.00,Ford,Focus,2024
928,NaN,2,8594.59,Ford,Focus,2020
929,NaN,2,3200.00,Ford,Focus,2020


####
## - Delete the last 20 observations from the `owners` dataframe and add three new observations that are not the same as those added to the `fines` dataframe.

In [10]:
print(len(owners))
owners = owners.iloc[:-20, :]
print(len(owners))
three_new = {
    'CarNumber': ['NEW_1NUM', 'NEW_2NUM', 'NEW_3NUM'],
    'SURNAME': ['Ivanov', 'Petrov', 'Sidorov']
}

three_new = pd.DataFrame(three_new)
owners = pd.concat([owners, three_new], ignore_index=True)
owners

531
511


,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
511,NEW_1NUM,Ivanov
512,NEW_2NUM,Petrov


####
## - Join the two dataframes.
     - The new dataframe should contain **only** the car numbers that exist in **both** dataframes.
     - The new dataframe should contain **all** the car numbers from **both** dataframes.
     - The new dataframe should contain **only** the car numbers from the `fines` dataframe.
     - The new dataframe should contain **only** the car numbers from the `owners` dataframe.

In [11]:
intersection_df = pd.merge(fines, owners, how='inner', on='CarNumber')
union_df = pd.merge(fines, owners, how='outer', on='CarNumber')
left_df = pd.merge(fines, owners, how='left', on='CarNumber')
right_df = pd.merge(fines, owners, how='right', on='CarNumber')

intersection_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,Y163O8161RUS,2,1600.00,Ford,Focus,1980,RICHARDSON
2,Y163O8161RUS,2,3200.00,Ford,Focus,2019,RICHARDSON
3,Y163O8161RUS,2,3200.00,Ford,Focus,2017,RICHARDSON
4,Y163O8161RUS,2,1600.00,Ford,Focus,2017,RICHARDSON
...,...,...,...,...,...,...,...
898,T914CT197RUS,2,7200.00,Toyota,Camry,2005,HERNANDEZ
899,E41977152RUS,2,2400.00,Ford,Focus,1989,BAKER
900,9464EX178RUS,2,2100.00,Ford,Focus,1988,MARTIN
901,O50197197RUS,2,7800.00,Ford,Focus,1992,WRIGHT


In [12]:
union_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1980.00,RICHARDSON
2,Y163O8161RUS,2.00,3200.00,Ford,Focus,2019.00,RICHARDSON
3,Y163O8161RUS,2.00,3200.00,Ford,Focus,2017.00,RICHARDSON
4,Y163O8161RUS,2.00,1600.00,Ford,Focus,2017.00,RICHARDSON
...,...,...,...,...,...,...,...
928,NaN,2.00,8594.59,Ford,Focus,2020.00,NaN
929,NaN,2.00,3200.00,Ford,Focus,2020.00,NaN
930,NEW_1NUM,NaN,NaN,NaN,NaN,NaN,Ivanov
931,NEW_2NUM,NaN,NaN,NaN,NaN,NaN,Petrov


In [13]:
left_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,NaN,1,500.00,Ford,Focus,2021,NaN
926,NaN,2,15200.00,Ford,Focus,2020,NaN
927,NaN,1,500.00,Ford,Focus,2024,NaN
928,NaN,2,8594.59,Ford,Focus,2020,NaN


In [14]:
right_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1980.00,RICHARDSON
2,Y163O8161RUS,2.00,3200.00,Ford,Focus,2019.00,RICHARDSON
3,Y163O8161RUS,2.00,3200.00,Ford,Focus,2017.00,RICHARDSON
4,Y163O8161RUS,2.00,1600.00,Ford,Focus,2017.00,RICHARDSON
...,...,...,...,...,...,...,...
901,O50197197RUS,2.00,7800.00,Ford,Focus,1992.00,WRIGHT
902,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2000.00,HILL
903,NEW_1NUM,NaN,NaN,NaN,NaN,NaN,Ivanov
904,NEW_2NUM,NaN,NaN,NaN,NaN,NaN,Petrov


####
## 5. Create a pivot table from the `fines` dataframe. It should look like this (the values are the sums of the fines), but with all the years. The values may be different for you:

In [15]:
pd.options.display.float_format = '{:,.2f}'.format
fines_pivot_df = pd.pivot_table(fines, values='Fines', index=['Make', 'Model'], columns='Year', aggfunc='sum') # можно добавить fill_value=0
fines_pivot_df

Year                    1980       1981       1982      1983       1984  \
Make       Model                                                          
Ford       Focus   62,394.59 395,589.17 140,383.76 63,100.00 111,294.59   
           Mondeo        NaN        NaN        NaN       NaN        NaN   
Skoda      Octavia 12,494.59        NaN   6,900.00 11,594.59   1,200.00   
Toyota     Camry   18,500.00   8,594.59        NaN  7,200.00        NaN   
           Corolla       NaN        NaN   2,000.00       NaN        NaN   
Volkswagen Golf    30,900.00        NaN        NaN  8,594.59     300.00   
           Jetta         NaN        NaN        NaN       NaN        NaN   
           Passat        NaN   4,600.00        NaN  3,200.00  10,000.00   
           Touareg       NaN        NaN        NaN       NaN        NaN   

Year                     1985      1986       1987      1988       1989  ...  \
Make       Model                                                         ...   
Ford       Focus   189,583.76 88,994.59 121,900.00 95,989.17 115,500.00  ...   
           Mondeo         NaN       NaN        NaN       NaN   8,600.00  ...   
Skoda      Octavia  10,294.59    600.00  26,700.00       NaN  91,400.00  ...   
Toyota     Camry          NaN       NaN        NaN       NaN  22,400.00  ...   
           Corolla        NaN 16,000.00   8,000.00       NaN   4,000.00  ...   
Volkswagen Golf     24,000.00       NaN   9,300.00       NaN   8,100.00  ...   
           Jetta          NaN       NaN        NaN       NaN        NaN  ...   
           Passat    5,000.00 15,000.00  12,300.00       NaN        NaN  ...   
           Touareg   5,800.00       NaN        NaN       NaN        NaN  ...   

Year                     2013       2014       2015      2016       2017  \
Make       Model                                                           
Ford       Focus   145,494.59 158,894.59 203,694.59 72,594.59 272,200.00   
           Mondeo         NaN        NaN        NaN 46,200.00        NaN   
Skoda      Octavia  13,794.59   1,900.00  46,394.59    300.00        NaN   
Toyota     Camry          NaN        NaN        NaN 13,000.00   1,000.00   
           Corolla        NaN        NaN        NaN       NaN   9,600.00   
Volkswagen Golf      9,300.00        NaN   2,300.00       NaN        NaN   
           Jetta          NaN        NaN        NaN       NaN        NaN   
           Passat      300.00        NaN   9,194.59  2,100.00        NaN   
           Touareg        NaN   1,300.00     500.00       NaN        NaN   

Year                     2018       2019      2020   2021   2024  
Make       Model                                                  
Ford       Focus   285,194.59 101,100.00 26,994.59 500.00 500.00  
           Mondeo         NaN        NaN       NaN    NaN    NaN  
Skoda      Octavia 156,200.00   9,500.00       NaN    NaN    NaN  
Toyota     Camry    13,000.00  18,100.00       NaN    NaN    NaN  
           Corolla        NaN  16,000.00       NaN    NaN    NaN  
Volkswagen Golf           NaN        NaN       NaN    NaN    NaN  
           Jetta          NaN        NaN       NaN    NaN    NaN  
           Passat         NaN        NaN       NaN    NaN    NaN  
           Touareg        NaN        NaN       NaN    NaN    NaN  

[9 rows x 43 columns]

####
## 6. Save both the `fines` and `owners` dataframes to CSV files without an index:

In [16]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)

In [17]:
# test_fines = pd.read_csv('../data/fines.csv')
# test_fines

In [18]:
# test_owners = pd.read_csv('../data/owners.csv')
# test_owners

In [19]:
fines.count()

CarNumber    925
Refund       930
Fines        930
Make         930
Model        919
Year         930
dtype: int64

In [20]:
owners.count()

CarNumber    514
SURNAME      514
dtype: int64

In [21]:
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
dtype: int64